In [1]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model
import numpy as np
import time
import config as cfg

begin = time.strftime('%Y-%m-%d_%H-%M-%S')

LOGDIR = './save'

tf.logging.set_verbosity(tf.logging.ERROR)

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

#start_learning_rate = 0.5e-3  # 1e-3
start_learning_rate = 0.5e-2  # 1e-3
#adjust_learning_rate = 1e-5
adjust_learning_rate = 1e-3

#onehot_labels = tf.one_hot(indices=tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth=4)
onehot_labels = tf.one_hot(indices = tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth = cfg.NUM_KEYS)

loss = tf.losses.softmax_cross_entropy( onehot_labels = onehot_labels, logits = model.y)
# loss = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

loss_val = tf.losses.softmax_cross_entropy( onehot_labels = onehot_labels, logits = model.y)
# loss_val = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
# train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

# sess.run(tf.initialize_all_variables())
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
# tf.scalar_summary("loss", loss)
tf.summary.scalar("loss", loss)

tf.summary.scalar("loss_val", loss_val)

# merge all summaries into a single op
# merged_summary_op = tf.merge_all_summaries()
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = tf.train.SaverDef.V2)
# saver.restore(sess, "save/model.ckpt")

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

#epochs = 10   
epochs = 30 
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data.num_images/batch_size)):
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        # print('Each item in xs : shape is...')
        # print(xs[0].shape)
        
        # train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.7})
        loss_value = loss.eval(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0})
        
        # print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))
        print("Epoch: %d, Step: %d, Loss: %g" % (epoch, i, loss_value))

        if i % 10 == 0:
            xs_val, ys_val = driving_data.LoadValBatch(batch_size)
            # xs, ys = driving_data.LoadTrainBatch(batch_size)
            loss_val = loss.eval(feed_dict={model.x:xs_val, model.y_: ys_val, model.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss_val: %g" % (epoch, i, loss_val))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
            print("Model saved in file: %s" % filename)


correct_prediction = tf.equal(tf.argmax(onehot_labels, 1), tf.argmax(model.y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Train Accuracy:', sess.run(accuracy, feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0}))
print('Validation Accuracy:', sess.run(accuracy, feed_dict={model.x: xs_val, model.y_: ys_val, model.keep_prob: 1.0}))

end = time.strftime('%Y-%m-%d_%H-%M-%S')
print('begin: ', begin)
print('end: ', end)

print("Run the command line:\n",
      "--> tensorboard --logdir=./logs --port=6006",
      "\nThen open http://0.0.0.0:6006/ into your web browser")

# os.system("sudo -s shutdown -h now")



BIAS가 없는 ANN 구조가 더 좋은 결과를 보이는 것 같은데... 확인해 보자

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
model read
Epoch: 0, Step: 0, Loss: 6.53561
Epoch: 0, Step: 0, Loss_val: 7.90407
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 1, Loss: 1.86846
Epoch: 0, Step: 2, Loss: 1.7892
Epoch: 0, Step: 3, Loss: 1.7587
Epoch: 0, Step: 4, Loss: 1.78421
Epoch: 0, Step: 5, Loss: 1.73489
Epoch: 0, Step: 6, Loss: 1.77491
Epoch: 0, Step: 7, Loss: 1.77333
Epoch: 0, Step: 8, Loss: 1.784
Epoch: 0, Step: 9, Loss: 1.771
Epoch: 0, Step: 10, Loss: 1.76771
Epoch: 0, Step: 10, Loss_val: 1.7717
Epoch: 0, Step: 11, Loss: 1.75719
Epoch: 0, Step: 12, Loss: 1.7839
Epoch: 0, Step: 13, Loss: 1.71067
Epoch: 0, Step: 14, Loss: 1.77142
Epoch: 0, Step: 15, Loss: 1.74493
Epoch: 0, Step: 16, Loss: 1.76949
Epoch: 0, Step: 17, Loss: 1.74473
Epoch: 0, Step: 18, Loss: 1.73196
Epoch: 0, Step: 19, Loss: 1.74989
Epoch: 0, Step: 20, Loss: 1.73051
Epoch: 0, Step: 

Epoch: 0, Step: 196, Loss: 1.11626
Epoch: 0, Step: 197, Loss: 0.953569
Epoch: 0, Step: 198, Loss: 1.13295
Epoch: 0, Step: 199, Loss: 1.0153
Epoch: 0, Step: 200, Loss: 1.14065
Epoch: 0, Step: 200, Loss_val: 1.16503
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 201, Loss: 1.01669
Epoch: 0, Step: 202, Loss: 1.01258
Epoch: 0, Step: 203, Loss: 1.05397
Epoch: 0, Step: 204, Loss: 1.11447
Epoch: 0, Step: 205, Loss: 1.06947
Epoch: 0, Step: 206, Loss: 1.15805
Epoch: 0, Step: 207, Loss: 1.05023
Epoch: 0, Step: 208, Loss: 1.09778
Epoch: 0, Step: 209, Loss: 1.16589
Epoch: 0, Step: 210, Loss: 1.10818
Epoch: 0, Step: 210, Loss_val: 1.16749
Epoch: 0, Step: 211, Loss: 1.23038
Epoch: 0, Step: 212, Loss: 1.22694
Epoch: 0, Step: 213, Loss: 1.2679
Epoch: 0, Step: 214, Loss: 1.32739
Epoch: 0, Step: 215, Loss: 1.24067
Epoch: 0, Step: 216, Loss: 1.02343
Epoch: 0, Step: 217, Loss: 1.04252
Epoch: 0, Step: 218, Loss: 1.0493
Epoch: 0, Step: 219, Loss: 1.21004
Epoch: 0, Step: 220, Loss: 1.05409
Epoch: 0, 

Epoch: 1, Step: 1, Loss: 0.971075
Epoch: 1, Step: 2, Loss: 0.966458
Epoch: 1, Step: 3, Loss: 1.07182
Epoch: 1, Step: 4, Loss: 1.01513
Epoch: 1, Step: 5, Loss: 1.12618
Epoch: 1, Step: 6, Loss: 1.07587
Epoch: 1, Step: 7, Loss: 1.139
Epoch: 1, Step: 8, Loss: 1.00496
Epoch: 1, Step: 9, Loss: 1.14921
Epoch: 1, Step: 10, Loss: 0.957679
Epoch: 1, Step: 10, Loss_val: 1.093
Epoch: 1, Step: 11, Loss: 1.19119
Epoch: 1, Step: 12, Loss: 1.22139
Epoch: 1, Step: 13, Loss: 1.05468
Epoch: 1, Step: 14, Loss: 1.08408
Epoch: 1, Step: 15, Loss: 1.18252
Epoch: 1, Step: 16, Loss: 1.06435
Epoch: 1, Step: 17, Loss: 1.24387
Epoch: 1, Step: 18, Loss: 1.15826
Epoch: 1, Step: 19, Loss: 1.0244
Epoch: 1, Step: 20, Loss: 1.03872
Epoch: 1, Step: 20, Loss_val: 1.12641
Epoch: 1, Step: 21, Loss: 1.0889
Epoch: 1, Step: 22, Loss: 1.11135
Epoch: 1, Step: 23, Loss: 1.13623
Epoch: 1, Step: 24, Loss: 0.972133
Epoch: 1, Step: 25, Loss: 0.929336
Epoch: 1, Step: 26, Loss: 1.01361
Epoch: 1, Step: 27, Loss: 0.999078
Epoch: 1, Step:

Epoch: 1, Step: 214, Loss: 1.17813
Epoch: 1, Step: 215, Loss: 1.18459
Epoch: 1, Step: 216, Loss: 1.11114
Epoch: 1, Step: 217, Loss: 1.126
Epoch: 1, Step: 218, Loss: 1.17409
Epoch: 1, Step: 219, Loss: 1.24285
Epoch: 1, Step: 220, Loss: 1.33026
Epoch: 1, Step: 220, Loss_val: 1.29102
Epoch: 1, Step: 221, Loss: 1.34718
Epoch: 1, Step: 222, Loss: 1.26013
Epoch: 1, Step: 223, Loss: 1.34858
Epoch: 1, Step: 224, Loss: 1.45149
Epoch: 1, Step: 225, Loss: 1.38662
Epoch: 1, Step: 226, Loss: 1.35498
Epoch: 1, Step: 227, Loss: 1.35899
Epoch: 1, Step: 228, Loss: 1.36167
Epoch: 1, Step: 229, Loss: 1.29479
Epoch: 1, Step: 230, Loss: 1.19747
Epoch: 1, Step: 230, Loss_val: 1.19991
Epoch: 1, Step: 231, Loss: 1.20926
Epoch: 1, Step: 232, Loss: 1.37902
Epoch: 1, Step: 233, Loss: 1.32845
Epoch: 1, Step: 234, Loss: 1.37443
Epoch: 1, Step: 235, Loss: 1.24546
Epoch: 1, Step: 236, Loss: 1.45062
Epoch: 1, Step: 237, Loss: 1.24517
Epoch: 1, Step: 238, Loss: 1.23402
Epoch: 1, Step: 239, Loss: 1.27756
Epoch: 1, Step

Epoch: 2, Step: 20, Loss_val: 1.28774
Epoch: 2, Step: 21, Loss: 1.3166
Epoch: 2, Step: 22, Loss: 1.34116
Epoch: 2, Step: 23, Loss: 1.20195
Epoch: 2, Step: 24, Loss: 1.2446
Epoch: 2, Step: 25, Loss: 1.17815
Epoch: 2, Step: 26, Loss: 1.08501
Epoch: 2, Step: 27, Loss: 1.20072
Epoch: 2, Step: 28, Loss: 1.36338
Epoch: 2, Step: 29, Loss: 1.31456
Epoch: 2, Step: 30, Loss: 1.27209
Epoch: 2, Step: 30, Loss_val: 1.35938
Epoch: 2, Step: 31, Loss: 1.39132
Epoch: 2, Step: 32, Loss: 1.2866
Epoch: 2, Step: 33, Loss: 1.27169
Epoch: 2, Step: 34, Loss: 1.27802
Epoch: 2, Step: 35, Loss: 1.15077
Epoch: 2, Step: 36, Loss: 1.17122
Epoch: 2, Step: 37, Loss: 1.03795
Epoch: 2, Step: 38, Loss: 1.2538
Epoch: 2, Step: 39, Loss: 1.14466
Epoch: 2, Step: 40, Loss: 1.2356
Epoch: 2, Step: 40, Loss_val: 1.31317
Epoch: 2, Step: 41, Loss: 1.1512
Epoch: 2, Step: 42, Loss: 1.11412
Epoch: 2, Step: 43, Loss: 1.10295
Epoch: 2, Step: 44, Loss: 1.18285
Epoch: 2, Step: 45, Loss: 1.10963
Epoch: 2, Step: 46, Loss: 1.22665
Epoch: 2

...

Epoch: 27, Step: 336, Loss: 1.64952
Epoch: 27, Step: 337, Loss: 1.66162
Epoch: 27, Step: 338, Loss: 1.65225
Epoch: 27, Step: 339, Loss: 1.62051
Epoch: 27, Step: 340, Loss: 1.625
Epoch: 27, Step: 340, Loss_val: 1.59722
Epoch: 27, Step: 341, Loss: 1.61633
Epoch: 27, Step: 342, Loss: 1.5333
Epoch: 27, Step: 343, Loss: 1.56218
Epoch: 27, Step: 344, Loss: 1.52541
Epoch: 27, Step: 345, Loss: 1.55832
Epoch: 27, Step: 346, Loss: 1.57932
Epoch: 27, Step: 347, Loss: 1.55996
Epoch: 27, Step: 348, Loss: 1.63106
Epoch: 27, Step: 349, Loss: 1.58003
Epoch: 27, Step: 350, Loss: 1.70531
Epoch: 27, Step: 350, Loss_val: 1.64201
Epoch: 27, Step: 351, Loss: 1.67708
Epoch: 27, Step: 352, Loss: 1.69175
Epoch: 27, Step: 353, Loss: 1.6742
Epoch: 27, Step: 354, Loss: 1.6807
Epoch: 27, Step: 355, Loss: 1.64617
Epoch: 27, Step: 356, Loss: 1.63557
Epoch: 27, Step: 357, Loss: 1.57749
Epoch: 27, Step: 358, Loss: 1.59413
Epoch: 27, Step: 359, Loss: 1.56085
Epoch: 27, Step: 360, Loss: 1.54023
Epoch: 27, Step: 360, Los

Epoch: 28, Step: 141, Loss: 1.52081
Epoch: 28, Step: 142, Loss: 1.51016
Epoch: 28, Step: 143, Loss: 1.56925
Epoch: 28, Step: 144, Loss: 1.56053
Epoch: 28, Step: 145, Loss: 1.49794
Epoch: 28, Step: 146, Loss: 1.57092
Epoch: 28, Step: 147, Loss: 1.59758
Epoch: 28, Step: 148, Loss: 1.5465
Epoch: 28, Step: 149, Loss: 1.49318
Epoch: 28, Step: 150, Loss: 1.54444
Epoch: 28, Step: 150, Loss_val: 1.49099
Epoch: 28, Step: 151, Loss: 1.54339
Epoch: 28, Step: 152, Loss: 1.55537
Epoch: 28, Step: 153, Loss: 1.55498
Epoch: 28, Step: 154, Loss: 1.50214
Epoch: 28, Step: 155, Loss: 1.583
Epoch: 28, Step: 156, Loss: 1.48854
Epoch: 28, Step: 157, Loss: 1.54048
Epoch: 28, Step: 158, Loss: 1.57822
Epoch: 28, Step: 159, Loss: 1.55556
Epoch: 28, Step: 160, Loss: 1.5705
Epoch: 28, Step: 160, Loss_val: 1.58933
Epoch: 28, Step: 161, Loss: 1.49138
Epoch: 28, Step: 162, Loss: 1.5482
Epoch: 28, Step: 163, Loss: 1.56187
Epoch: 28, Step: 164, Loss: 1.51262
Epoch: 28, Step: 165, Loss: 1.54567
Epoch: 28, Step: 166, Los

Epoch: 28, Step: 347, Loss: 1.625
Epoch: 28, Step: 348, Loss: 1.60457
Epoch: 28, Step: 349, Loss: 1.54887
Epoch: 28, Step: 350, Loss: 1.47644
Epoch: 28, Step: 350, Loss_val: 1.52427
Epoch: 28, Step: 351, Loss: 1.52117
Epoch: 28, Step: 352, Loss: 1.53807
Epoch: 28, Step: 353, Loss: 1.41438
Epoch: 28, Step: 354, Loss: 1.53237
Epoch: 28, Step: 355, Loss: 1.57774
Epoch: 28, Step: 356, Loss: 1.52887
Epoch: 28, Step: 357, Loss: 1.62058
Epoch: 28, Step: 358, Loss: 1.61024
Epoch: 28, Step: 359, Loss: 1.54269
Epoch: 28, Step: 360, Loss: 1.60564
Epoch: 28, Step: 360, Loss_val: 1.53006
Epoch: 28, Step: 361, Loss: 1.63915
Epoch: 28, Step: 362, Loss: 1.4924
Epoch: 28, Step: 363, Loss: 1.60572
Epoch: 28, Step: 364, Loss: 1.59868
Epoch: 28, Step: 365, Loss: 1.52881
Epoch: 28, Step: 366, Loss: 1.53263
Epoch: 28, Step: 367, Loss: 1.45864
Epoch: 28, Step: 368, Loss: 1.62876
Epoch: 28, Step: 369, Loss: 1.53743
Epoch: 28, Step: 370, Loss: 1.53811
Epoch: 28, Step: 370, Loss_val: 1.5553
Epoch: 28, Step: 371

Epoch: 29, Step: 150, Loss: 1.51458
Epoch: 29, Step: 150, Loss_val: 1.5444
Epoch: 29, Step: 151, Loss: 1.60248
Epoch: 29, Step: 152, Loss: 1.48963
Epoch: 29, Step: 153, Loss: 1.52655
Epoch: 29, Step: 154, Loss: 1.57443
Epoch: 29, Step: 155, Loss: 1.5763
Epoch: 29, Step: 156, Loss: 1.64263
Epoch: 29, Step: 157, Loss: 1.61813
Epoch: 29, Step: 158, Loss: 1.60933
Epoch: 29, Step: 159, Loss: 1.56746
Epoch: 29, Step: 160, Loss: 1.62903
Epoch: 29, Step: 160, Loss_val: 1.52569
Epoch: 29, Step: 161, Loss: 1.6513
Epoch: 29, Step: 162, Loss: 1.54639
Epoch: 29, Step: 163, Loss: 1.52316
Epoch: 29, Step: 164, Loss: 1.57562
Epoch: 29, Step: 165, Loss: 1.52726
Epoch: 29, Step: 166, Loss: 1.49859
Epoch: 29, Step: 167, Loss: 1.57117
Epoch: 29, Step: 168, Loss: 1.54956
Epoch: 29, Step: 169, Loss: 1.56779
Epoch: 29, Step: 170, Loss: 1.51067
Epoch: 29, Step: 170, Loss_val: 1.64429
Epoch: 29, Step: 171, Loss: 1.50719
Epoch: 29, Step: 172, Loss: 1.53381
Epoch: 29, Step: 173, Loss: 1.55468
Epoch: 29, Step: 17

Epoch: 29, Step: 355, Loss: 1.56286
Epoch: 29, Step: 356, Loss: 1.5003
Epoch: 29, Step: 357, Loss: 1.58709
Epoch: 29, Step: 358, Loss: 1.57069
Epoch: 29, Step: 359, Loss: 1.51484
Epoch: 29, Step: 360, Loss: 1.53985
Epoch: 29, Step: 360, Loss_val: 1.61777
Epoch: 29, Step: 361, Loss: 1.56197
Epoch: 29, Step: 362, Loss: 1.53255
Epoch: 29, Step: 363, Loss: 1.58118
Epoch: 29, Step: 364, Loss: 1.54302
Epoch: 29, Step: 365, Loss: 1.54044
Epoch: 29, Step: 366, Loss: 1.57789
Epoch: 29, Step: 367, Loss: 1.53924
Epoch: 29, Step: 368, Loss: 1.47462
Epoch: 29, Step: 369, Loss: 1.55585
Epoch: 29, Step: 370, Loss: 1.50955
Epoch: 29, Step: 370, Loss_val: 1.63088
Epoch: 29, Step: 371, Loss: 1.5218
Epoch: 29, Step: 372, Loss: 1.55038
Epoch: 29, Step: 373, Loss: 1.55103
Epoch: 29, Step: 374, Loss: 1.49439
Epoch: 29, Step: 375, Loss: 1.50292
Epoch: 29, Step: 376, Loss: 1.5077
Epoch: 29, Step: 377, Loss: 1.52749
Epoch: 29, Step: 378, Loss: 1.57735
Epoch: 29, Step: 379, Loss: 1.60048
Epoch: 29, Step: 380, L